<a href="https://colab.research.google.com/github/Pavansree14/Python/blob/Pyspark_advanced/tes1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip3 install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=926125e7d985e52b47cc4179b3d8f1b0562ffeb71e438283a889cf4cf7c1f79c
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [4]:
from pyspark import SparkContext, SparkConf

In [5]:
conf = SparkConf().setAppName("testApp1").setMaster("local[*]")
sc = SparkContext(conf=conf)

In [6]:
print(sc)

<SparkContext master=local[*] appName=testApp1>


In [7]:
import random
randomlist  = random.sample(range(0,40),10)
print(randomlist)

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]


In [8]:
rdd1 = sc.parallelize(randomlist, 4)

rdd1.collect()

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]

In [9]:
rdd1.getNumPartitions()

4

In [10]:
print(rdd1.glom().collect())

[[8, 29], [25, 24], [3, 38], [26, 37, 22, 28]]


In [11]:
rdd_map = rdd1.map(lambda item: (item + 1)*3)
rdd_map.collect()

[27, 90, 78, 75, 12, 117, 81, 114, 69, 87]

In [12]:
print(rdd_map.glom().collect())

[[27, 90], [78, 75], [12, 117], [81, 114, 69, 87]]


In [13]:
rdd_filter = rdd1.filter(lambda x: x%3 == 0)
rdd_filter.collect()

[24, 3]

In [14]:
(rdd_filter.glom().collect())

[[], [24], [3], []]

In [15]:
#union
print(rdd1.collect())

rdd2 = sc.parallelize([14,20,28,30,24,86,32,22],2)
rdd2.collect()
rdd_union = rdd1.union(rdd2)
print(rdd_union.collect())

print(rdd_union.getNumPartitions())

[8, 29, 25, 24, 3, 38, 26, 37, 22, 28]
[8, 29, 25, 24, 3, 38, 26, 37, 22, 28, 14, 20, 28, 30, 24, 86, 32, 22]
6


In [16]:
#Intersection
rdd_intersection = rdd1.intersection(rdd2)
print(rdd_intersection.collect())
print(rdd_intersection.getNumPartitions())
rdd_intersection.glom().collect()

[24, 22, 28]
6


[[24], [], [], [], [22, 28], []]

In [17]:
#find empty partitions
counter = 0
for item in rdd_intersection.glom().collect():
    if len(item) == 0:
      counter = counter + 1

counter


4

In [18]:
#coalesce(num of partitions)--reduce the size of partitions
rdd_intersection.coalesce(1).glom().collect()

[[24, 22, 28]]

In [19]:
#takesample(withReplacement, num, [seed])
rdd1.takeSample(False, 5)

[28, 37, 3, 24, 8]

In [20]:
#takeordered(n, [ordering])
print(rdd1.takeOrdered(5))
print(rdd1.takeOrdered(5, key = lambda x: -x))

[3, 8, 22, 24, 25]
[38, 37, 29, 28, 26]


In [21]:
rdd2.glom().collect()

[[14, 20, 28, 30], [24, 86, 32, 22]]

In [22]:
#reduce by key
rdd_rbk = sc.parallelize([(2,6),(7,5),(3,8),(6.8),(2,9),(5,3),(4,6),(6,9)], 2)

print(rdd_rbk.glom().collect())


[[(2, 6), (7, 5), (3, 8), 6.8], [(2, 9), (5, 3), (4, 6), (6, 9)]]


In [29]:
#reduce
rdd1.reduce(lambda x,y: x-y)

74

Query optimization
catalyst query optimization
Project tungsten.